In [54]:
# import QGRID para mejor visualizacion de los datos
# los pptos sin abonos se deben cancelar en ClinicCloud, cosa q no generen alertas. O q el paciente tenga saldo a favor
# Debemos Generar un archivo identificandolos. Las ff´s deben tener algun ingreso de dinero para no cancelarlas.
# Los pptos aceptados hoy pueden haber sido elaborados en una fecha anterior. Su fecha se debe actualizar a la de hoy, 
# ya sea en ClinicCloud o en el reporte (mejor - 'fecha_inicio')

import numpy as np
import pandas as pd
import time

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.set_option('display.max_colwidth', -1)

In [55]:
archivo_PPE = 'Produccion_pendiente_exigible_con_tratamientos_Dentix_Calle_90.csv'
archivo_crono = 'crono_Dentix_Calle_90_1.csv'

In [56]:
crono_df = pd.read_csv(archivo_crono, encoding = 'unicode_escape', sep=',')
PPE = pd.read_csv(archivo_PPE, encoding = 'unicode_escape', sep=',')

#Eliminación de Puntos y Revisiones del PPE
PPE.set_index('Servicio', inplace=True)
PPE.drop(index='Puntos y Revisiones.  ',inplace=True)
PPE.reset_index(inplace=True)
PPE['duplicated'] = PPE.duplicated()
PPE.set_index('duplicated',inplace=True)
PPE.drop(True,inplace=True)
PPE.reset_index(inplace=True)
PPE.drop(columns=['duplicated'],inplace=True)
PPE

,Servicio,Clínica,Fecha Pres.,Nº Historia,Presupuesto,Descuento,Teléfono Paciente,Doctor,Especialidad,Explicado,Importe Pres.,Pagado Pres.,% Pagado,% Realizado,Producidos,Pendientes Producción,Próxima Cita,Fecha Última Cita Prod
0,CITA TALLADO SELECTIVO X PIEZA. Pieza: 11,Dentix Calle 90,30/08/2018,BG058042,Nº 4119 - JULIAN ANDRES JIMENEZ,0,3143511778,LAURA ALEJANDRA RONCANCIO GARCIA,OCLUSION Y DISFUNCION TEMPOROMANDIBULAR,SANDRA PATRICIA GONZALEZ TOVAR,118720,0,0.00%,0.00%,0,118700,01/09/2018,Sin Cita Producida
1,CARILLA ESTETICA EN RESINA. Pieza: 12,Dentix Calle 90,30/08/2018,BG058042,Nº 4119 - JULIAN ANDRES JIMENEZ,0,3143511778,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,304950,0,0.00%,0.00%,0,304950,01/09/2018,Sin Cita Producida
2,CARILLA ESTETICA EN RESINA. Pieza: 22,Dentix Calle 90,30/08/2018,BG058042,Nº 4119 - JULIAN ANDRES JIMENEZ,0,3143511778,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,304950,0,0.00%,0.00%,0,304950,01/09/2018,Sin Cita Producida
3,RESINA FOTOPOLIMERIZADO 3 SUP POSTERIOR O POSTERIOR. Pieza: 11,Dentix Calle 90,30/08/2018,BG058042,Nº 4119 - JULIAN ANDRES JIMENEZ,0,3143511778,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,116500,0,0.00%,0.00%,0,116500,01/09/2018,Sin Cita Producida
4,RESINA FOTOPOLIMERIZADO 3 SUP POSTERIOR O POSTERIOR. Pieza: 21,Dentix Calle 90,30/08/2018,BG058042,Nº 4119 - JULIAN ANDRES JIMENEZ,0,3143511778,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,116500,0,0.00%,0.00%,0,116500,01/09/2018,Sin Cita Producida
5,BLANQUEAMIENTO DENTAL 2. Pieza: 11,Dentix Calle 90,30/08/2018,BG058042,Nº 4119 - JULIAN ANDRES JIMENEZ,0,3143511778,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,412500,0,0.00%,0.00%,0,412500,01/09/2018,Sin Cita Producida
6,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR. Pieza: 47,Dentix Calle 90,30/08/2018,BG058042,Nº 4119 - JULIAN ANDRES JIMENEZ,0,3143511778,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,79000,0,0.00%,0.00%,0,79000,01/09/2018,Sin Cita Producida
7,REPARACION ESTANDAR (REPARACION METALICOS). Pieza: 24,Dentix Calle 90,30/08/2018,BG055160,Nº 4126 - NIDIA MARCELA VENTERO BOHORQUEZ,0,3204932499,SANDRA PATRICIA GOMEZ MESA,ORTODONCIA Y REPARACIONES,SANDRA PATRICIA GONZALEZ TOVAR,21000,21000,100%,0.00%,0,21000,No Citado,Sin Cita Producida
8,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR. Pieza: 14,Dentix Calle 90,30/08/2018,BG054450,Nº 4131 - RICARDO ANDRES GUZMAN RINCON,10,3112178885,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,79000,0,0.00%,0.00%,0,71100,31/08/2018,Sin Cita Producida
9,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR. Pieza: 34,Dentix Calle 90,30/08/2018,BG054450,Nº 4131 - RICARDO ANDRES GUZMAN RINCON,10,3112178885,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,79000,0,0.00%,0.00%,0,71100,31/08/2018,Sin Cita Producida


In [57]:
print('REGISTROS CRONOGRAMA:',crono_df.count()[0])
print('REGISTROS PPE:',PPE.count()[0])

REGISTROS CRONOGRAMA: 59
REGISTROS PPE: 52


In [58]:
#CRUCE PPE - CRONO: en este punto revisar que cada ttmto del PPE tenga su cronograma. Sinó, sacar los pptos q faltan por crono.

pptos_faltantes = pd.merge(PPE,crono_df,how='left',left_on=['Clínica','Nº Historia','Presupuesto','Servicio'], right_on=['Clínica','Nº Historia','Presupuesto','Servicio'])
pptos_faltantes['Semana'].fillna(0,inplace=True)
pptos_faltantes.set_index('Semana',inplace=True)
pptos_faltantes = pptos_faltantes.loc[0]

pptos_faltantes

KeyError: 'the label [0] is not in the [index]'

In [ ]:
pptos_faltantes.reset_index(inplace=True)
pptos_faltantes['Cantidad Presupuestada'] = (pptos_faltantes['Producidos']+pptos_faltantes['Pendientes Producción'])/pptos_faltantes['Importe Pres.']
pptos_faltantes['Cantidad Presupuestada'] = np.ceil(pptos_faltantes['Cantidad Presupuestada'])
pptos_faltantes['Cantidad Presupuestada'].fillna(1,inplace=True) #las CSI traen vacío el campo 'Pendientes Produccion'.
# en realidad parece que las unicas cantidades presupuestadas q son necesarias son las de Mensualidades de ortodoncia.
# Para CSI, el %Realizado es el que nos sirve para calcular Cantidad Producida.
# la Cantidad Pendiente se puede utilizar para saber la fecha de colocación de un implante si hay ortodoncia.

pptos_faltantes['ttmto_homol'] = pptos_faltantes['Servicio'].str.slice(0,-13)
pptos_faltantes['ttmto_homol'] = pptos_faltantes['ttmto_homol'].str.strip()
pptos_faltantes['pieza'] = pptos_faltantes['Servicio'].str.slice(-2)
pptos_faltantes['pieza'] = pptos_faltantes['pieza'].str.strip()
pptos_faltantes['fecha_inicio'] = time.strftime("%d/%m/%Y")

pptos_faltantes = pptos_faltantes[['Clínica', 'Fecha Pres._x', 'Nº Historia', 'Presupuesto', 
                                   'Servicio', 'Semana', 'Cantidad Presupuestada','Doctor_x', 
                                   'Explicado_x', 'ttmto_homol','pieza', 'fecha_inicio']]

print('Numero de tratamientos faltantes por crono:',pptos_faltantes.count()[0])
pptos_faltantes.to_csv('pptos_faltantes.csv',index = False)

pptos_faltantes.set_index(['Clínica','Nº Historia','Servicio'], inplace=True)
pptos_faltantes